# Intro

In [20]:
from tinygrad import Tensor, Context
a = Tensor.empty(4,4)
b = Tensor.empty(4,4)

print((a+b).tolist())

[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]


In [21]:
print(a+b)

<Tensor <UOp NV (4, 4) float> on NV with grad None>


Lazy computation so only computed answer if tolist numpy or realize on tensor are used

In [22]:
with Context(DEBUG=4): 
    a = Tensor.empty(4,4)
    b = Tensor.empty(4,4)
    print((a+b).tolist())

scheduled 1 kernels in 0.90 ms
*** NV         5 E_4_4                                          arg  3 mem   0.00 GB tm      1.50us/     0.01ms (      0 GFLOPS    0|0      GB/s) ['__add__']
[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]
